In [16]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
from IPython.display import HTML

def create_trinomial_animation(a=1, b=1.2, c=1.5, fps=5, width_cm=14.3, height_cm=25.4, y_offset=-1.5):
    L = a + b + c
    pause_frames = fps

    colors = {
        r'$a^2$': '#e63946cc',
        r'$b^2$': '#457b9dcc',
        r'$c^2$': '#2a9d8fcc',
        r'$ab$':  '#9b5de5cc',
        r'$ac$':  '#f4a261cc',
        r'$bc$':  '#e9c46acc',
    }

    ordered_regions = [
        (r'$a^2$', (0, L - a + y_offset), a, a),
        (r'$b^2$', (a, L - a - b + y_offset), b, b),
        (r'$c^2$', (a + b, 0 + y_offset), c, c),
        (r'$ab$', (0, L - a - b + y_offset), a, b),
        (r'$ab$', (a, L - a + y_offset), b, a),
        (r'$ac$', (0, 0 + y_offset), a, c),
        (r'$ac$', (a + b, L - a + y_offset), c, a),
        (r'$bc$', (a + b, L - a - b + y_offset), c, b),
        (r'$bc$', (a, 0 + y_offset), b, c)
    ]

    appearance_schedule = [i * pause_frames for i in range(len(ordered_regions))]
    eq_frame = appearance_schedule[-1] + pause_frames
    total_frames = eq_frame + pause_frames + 10

    fig, ax = plt.subplots(figsize=(width_cm / 2.54, height_cm / 2.54))
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_xlim(0, L)
    ax.set_ylim(-1.2, L + 0.6)

    patches_list = []

    def init():
        return []

    def animate_spaced(i):
        artists = []

        if i == 0:
            eq_main = r'$(a + b + c)^2 = a^2 + b^2 + c^2 + 2ab + 2ac + 2bc$'
            eq_text = ax.text(L / 2, L + 0.3, eq_main, ha='center', fontsize=16, color='white')
            artists.append(eq_text)

        for idx, frame_start in enumerate(appearance_schedule):
            if i >= frame_start:
                label, (x, y), w, h = ordered_regions[idx]
                if idx >= len(patches_list):
                    face_color = colors[label]
                    rect = patches.Rectangle((x, y), w, h, linewidth=2.5, edgecolor='white',
                                             facecolor=face_color, alpha=0.0)
                    ax.add_patch(rect)
                    txt = ax.text(x + w / 2, y + h / 2, label, va='center', ha='center',
                                  fontsize=14, color='white', alpha=0.0)

                    def fade(k, rect=rect, txt=txt, frame_start=frame_start):
                        alpha = max(0.0, min(1.0, (k - frame_start) / 8))
                        rect.set_alpha(alpha)
                        txt.set_alpha(alpha)
                        return [rect, txt]

                    patches_list.append((rect, txt, fade))

                rect, txt, fade_func = patches_list[idx]
                artists += fade_func(i)

        return artists

    ani = animation.FuncAnimation(fig, animate_spaced, init_func=init,
                                  frames=total_frames, interval=1000 // fps, blit=False)
    plt.close(fig)  
    return ani, fig

In [19]:
fps = 4
ani, fig = create_trinomial_animation(a=1, b=1.2, c=1.5,fps=fps,width_cm=14.3,height_cm=25.4,y_offset=-1)
HTML(ani.to_jshtml())


In [7]:
ani, fig = create_trinomial_animation()
ani.save("trinomial_square.mp4", writer=FFMpegWriter(fps=fps), dpi=500)
#plt.close(fig)